# Taask 4.2 Demonstrate running Circuits on Real Hardware

## Objective 1 : Execute on Target Hardware

### Overview

Executing quantum circuits on real hardware involves several key steps:

1. **Authentication**: Connecting to the IBM Quantum Runtime service
2. **Backend Selection**: Choosing an available quantum processor
3. **Circuit Preparation**: Transpiling circuits for specific hardware constraints
4. **Job Submission**: Sending circuits for execution
5. **Result Retrieval**: Obtain and analyze measurement outcomes

<span style="color:red"> **Running the cell below will use around 10-15 seconds from your IBM account credit**
</span>

In [ ]:
# Import necessary libraries
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit.quantum_info import SparsePauliOp

# --- STEP 1: Authentication ---
# if account is already saved, this will load it
service = QiskitRuntimeService()
# replace <instance_name> with your instance name
# replace <token> with your IBM Quantum API token
# Uncomment and use the following line if you need to specify instance and token
# srvice = QiskitRuntimeService(channel="ibm_cloud",instance="<instance_name>",token=<token>)

   
# --- STEP 2: Backend Selection ---
# Choose the least busy available backend to minimize queue time with real hardware only
backend = service.least_busy(operational=True, simulator=False)
print(f"Using backend: {backend.name}")
print(f"Backend properties: {backend.num_qubits} qubits")

# --- STEP 3: Circuit Preparation ---
# Create a simple Bell state circuit
qc = QuantumCircuit(2, 2)
qc.h(0)                   
qc.cx(0, 1) 
qc.measure([0, 1], [0, 1])

# Display the circuit
qc.draw('mpl')

# Transpile the circuit for the specific hardware backend
# This transpiles the circuit to the backend's: Native gate set,  Qubit connectivity and Optimization level (0-3, where 3 is most optimized)
pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
transpiled_circuit = pm.run(qc)

# Define the observable we want to measure (ZZ correlation for example)
ZZ = SparsePauliOp.from_list([("ZZ", 1)])

# --- STEP 4: Job Submission ---
# Create an Estimator instance and run the job
estimator = Estimator(mode=backend)

# Map observables to the transpiled circuit layout
observables = [ZZ.apply_layout(transpiled_circuit.layout)]

# Submit the job
job = estimator.run([(transpiled_circuit, observables)])
print(f"\nJob ID: {job.job_id()}")
print("Job submitted successfully! Check the IBM Quantum dashboard for status.")

In [ ]:
# --- STEP 5: Result Retrieval ---
result = job.result()
print(f"Expectation value: {result[0].data.evs}")

## Objective 2: Understanding Primitives

Primitives are foundational quantum computing operations that abstract away hardware-specific details. They provide standardized interfaces for common quantum tasks.

### Key Primitives:


**1. Sampler**:
- **Used for**: Sampling measurement outcomes from quantum circuits
- **Output**: Counts or probability distributions

**2. Estimator**:
- **Used for**: Calculating expectation values of observables
- **Output**: Expectation value ± standard error

### Primitive Implementations:

**Base Classes**:
- `BaseSamplerV2` : Base class for sampler, all sampler implementations inherit from this class
- `BaseEstimatorV2`: Base class for estimator, all estimator implementations inherit from this class

**V2 Primitives**:
- `SamplerV2`: Latest sampler with enhanced capabilities
- `EstimatorV2`: Latest estimator with improved performance and features

**Alternatives**:
- `StateVectorSampler`: Simulator-based sampler
- `StateVectorEstimator`: Simulator-based estimator using state vector simulation
- `BackendSamplerV2`: Backend-specific sampler optimizations
- `BackendEstimatorV2`: Backend-specific estimator optimizations


<span style="color:red"> **Running the cell below will use around 5-10 seconds from your IBM account credit** </span>

In [ ]:
# Example: Using the Sampler Primitive 
# The Sampler primitive executes quantum circuits and returns measurement statistics.

from qiskit_ibm_runtime import SamplerV2 as Sampler

# Create a Sampler instance connected to our backend
sampler = Sampler(mode=backend)

# The 'shots' parameter determines how many times to execute the circuit
shots = 1024

# Submit the sampling job with the transpiler bell state circuit
job = sampler.run([transpiled_circuit], shots=shots)

print(f"Sampler Job ID: {job.job_id()}")
print(f"Requested shots: {shots}")

In [ ]:
# To retrieve results:
result = job.result()
counts = result[0].data.c.get_counts()
print(f"\nMeasurement counts: {counts}")

## Objective 3: Primitive Inputs and Outputs

### Inputs

#### Sampler Input Structure

Each Sampler **Pub** (Primitive Unified Bloc) accepts:
1. **Quantum Circuit**: The circuit to execute (should include measurements for qubits that will be sampled)
2. **Parameters**: Parameter values for parameterized circuits (needed only if the circuit is parameterized)
3. **Shots**: Number of circuit executions (optional)

#### Estimator Input Structure

Each Estimator **Pub**  accepts:
1. **Quantum Circuit**: The circuit to execute (must be transpiled for the target backend)
2. **Observables**: One or more observables to measure (as `Pauli`, `SparsePauliOp`, `PauliList` or `str` objects)
3. **Parameters**: Parameter values for parameterized circuits (optional)
4. **Precision**: Target precision for error mitigation (optional)



<span style="color:red"> **Running the cell below will use around 5 minutes from your IBM account credit**
</span>

In [ ]:
# Example 1: Using the Estimator Primitive with Multiple Parameters and Observables

from qiskit.circuit import Parameter
from qiskit import QuantumCircuit
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp
from qiskit_ibm_runtime import EstimatorV2 as Estimator
import numpy as np

# --- Create a parameterized circuit ---
circuit = QuantumCircuit(2)
circuit.h(0)                     
circuit.cx(0, 1)                 
circuit.ry(Parameter("a"), 0)    # Parameterized rotation Y
circuit.rz(Parameter("b"), 0)    # Parameterized rotation Z
circuit.cx(0, 1)                 
circuit.h(0)

# Display the parameterized circuit
print("Parameterized Circuit:")
circuit.draw('mpl')

# --- Transpile for hardware ---
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
transpiled_circuit = pm.run(circuit)
layout = transpiled_circuit.layout

# --- Define parameter sweep ---
# Create 100 different parameter combinations
# Shape: (100, 2) where 100 is number of parameter sets, 2 is parameters per set
params = np.vstack([
    np.linspace(-np.pi, np.pi, 100),        # Parameter 'a' values
    np.linspace(-4 * np.pi, 4 * np.pi, 100) # Parameter 'b' values
]).T

print(f"\nParameter array shape: {params.shape}")
print(f"First 5 parameter sets:\n{params[:5]}")

# --- Define multiple observables ---
# measure three different observables for each parameter set
# Shape: (3, 1) where 3 is number of observable sets, 1 is observables per set
observables = [
    [SparsePauliOp(["XX", "IY"], [0.5, 0.5])],   # Linear combination of XX and IY
    [SparsePauliOp("XX")],                       # XX
    [SparsePauliOp("IY")],                       # IY
]

# Apply the same layout as the transpiled circuit
observables = [
    [observable.apply_layout(layout) for observable in observable_set]
    for observable_set in observables
]

print(f"\nNumber of observable sets: {len(observables)}")
print(f"Observables per set: {len(observables[0])}")

# --- Create the Estimator Pub ---
# Total calculations: 1 circuit  × 3 observables × 100 parameter sets = 300 expectation values
estimator_pub = (transpiled_circuit, observables, params)

# --- Execute with Estimator ---
estimator = Estimator(mode=backend)
job = estimator.run([estimator_pub])

print(f"\nJob submitted! Job ID: {job.job_id()}")

In [ ]:

# --- Process Results ---
result = job.result()
print(f"\nResult shape: {result[0].data.evs.shape}")  # Should be (3, 100)
print(f"First expectation values: {result[0].data.evs[:, :5]}")

#### Broadcasting Rules

Broadcasting allows you to efficiently compute expectation values for multiple parameter sets and observables in a single primitive call. The rules follow **NumPy broadcasting conventions**.

##### Broadcasting Concepts:

1. **Basic Input Components**:
   - **Circuits**: Can be single circuits or lists of circuits
   - **Observables**: Can be single observables or nested lists of observables
   - **Parameters**: Can be single parameter sets or arrays of parameter sets

2. **Broadcasting Dimensions**:
   - **Circuit dimension**: Outer-most dimension (if multiple circuits)
   - **Observable dimension**: Middle dimension (if multiple observables per circuit)
   - **Parameter dimension**: Inner-most dimension (if multiple parameter sets)



##### Broadcasting Rules:


- Input arrays do not need to have the same number of dimensions.
   - The resulting array will have the same number of dimensions as the input array with the largest dimension.
   - The size of each dimension is the largest size of the corresponding dimension.
   - Missing dimensions are assumed to have size one.
- Shape comparisons start with the rightmost dimension and continue to the left.
- Two dimensions are compatible if their sizes are equal or if one of them is 1.

1. **Single Circuit, Multiple Observables, Multiple Parameters**:
   ```python
   # Shape: (1 circuit, 3 observables, 100 parameter sets)
   # Result shape: (3, 100)
   estimator.run([(circuit, [[obs1], [obs2], [obs3]], params)])
   ```

2. **Multiple Circuits, Single Observable, Multiple Parameters**:
   ```python
   # Shape: (3 circuits, 1 observable, 100 parameter sets)
   # Result shape: (3, 100)
   estimator.run([(circuit1, observable, params), 
                  (circuit2, observable, params), 
                  (circuit3, observable, params)])
   ```

3. **Parameter Broadcasting**:
   ```python
   # Parameters broadcast across observables
   # Shape: (1 circuit, 3 observables, 100 parameter sets)
   estimator.run([(circuit, [[obs1], [obs2], [obs3]], params)])
   ```

##### Example Patterns:

```python
# Pattern 1: Single circuit, single observable, multiple parameters
# Result shape: (1, 100)
estimator.run([(circuit, observable, params)])

# Pattern 2: Single circuit, multiple observables, single parameter set
# Result shape: (3, 1)
estimator.run([(circuit, [[obs1], [obs2], [obs3]], single_params)])

# Pattern 3: Multiple circuits, multiple observables, multiple parameters
# Result shape: (2 circuits × 3 observables × 100 params = 600 values)
estimator.run([
    (circuit1, [[obs1], [obs2], [obs3]], params),
    (circuit2, [[obs1], [obs2], [obs3]], params)
])
```

##### Important Notes:

- **Inner lists matter**: `[[obs1], [obs2]]` has shape (2, 1) while `[obs1, obs2]` has shape (2,)
- **Result shape**: Follows the broadcast shape of inputs
- **Efficiency**: Broadcasting reduces overhead compared to multiple separate calls

##### Practical Example from code above:

```python
# Our example: 1 circuit × 3 observables × 100 parameter sets
estimator_pub = (transpiled_circuit, observables, params)
# observables shape: (3, 1)  # 3 observable sets, 1 observable per set
# params shape: (100, 2)     # 100 parameter sets, 2 parameters per set
# Result shape: (3, 100)     # 3 observables × 100 parameter sets
```

This broadcasting mechanism allows efficient computation of multiple quantum expectation values in a single job submission, reducing queue times and overhead.

### Outputs


the data returned from the result object is a `PubResult` list, each element in this list corresponds to each PUB submitted as an input to the primitive `run` method 

#### Sampler Output Structure:

```
└── PrimitiveResult
    ├── PubResult[0]
    │   ├── metadata
    │   └── data  ## In the form of a DataBin object
    │       ├── NAME_OF_CLASSICAL_REGISTER
    │       │   └── BitArray of count data (default is 'meas')
    |       |
    │       └── NAME_OF_ANOTHER_CLASSICAL_REGISTER
    │           └── BitArray of count data (exists only if more than one
    |                 ClassicalRegister was specified in the circuit)
    ├── PubResult[1]
    |   ├── metadata
    |   └── data  ## In the form of a DataBin object
    |       └── NAME_OF_CLASSICAL_REGISTER
    |           └── BitArray of count data for second pub
    ├── ...
    ├── ...
    └── ...
```

##### Estimator Output Structure:

```
└── PrimitiveResult
    ├── PubResult[0]
    │   ├── metadata
    │   └── data  ## In the form of a DataBin object
    │       ├── evs
    │       │   └── List of estimated expectation values in the shape
    |       |         specified by the first pub
    │       └── stds
    │           └── List of calculated standard deviations in the
    |                 same shape as above
    ├── PubResult[1]
    |   ├── metadata
    |   └── data  ## In the form of a DataBin object
    |       ├── evs
    |       │   └── List of estimated expectation values in the shape
    |       |        specified by the second pub
    |       └── stds
    |           └── List of calculated standard deviations in the
    |                same shape as above
    ├── ...
    ├── ...
    └── ...
```

<span style="color:red"> **Running the cell below will use around 5 seconds from your IBM account credit**
</span>

In [ ]:
# Example 2: Using the Sampler Primitive and Processing Results

from qiskit_ibm_runtime import SamplerV2 as Sampler

# --- Create a 10-qubit GHZ circuit ---
# GHZ states are maximally entangled states: (|0...0⟩ + |1...1⟩)/√2
circuit = QuantumCircuit(10)
circuit.h(0)                        
circuit.cx(range(0, 9), range(1, 10)) 
circuit.measure_all()                

print("GHZ State Circuit (10 qubits):")
circuit.draw('mpl', fold=-1)  # Don't fold the circuit display

# --- Transpile for hardware ---
transpiled_circuit = pm.run(circuit)
sampler_pub = [transpiled_circuit]
# --- Execute with Sampler ---
sampler = Sampler(mode=backend)
job = sampler.run(sampler_pub, shots=1000)
print(f"Sampler Job ID: {job.job_id()}")

In [ ]:
result = job.result()

print("\n=== SAMPLER RESULTS ANALYSIS ===\n")

# 1. Access the data bin
data = result[0].data
print(f"DataBin contents: {data}")
print(f"Available registers: {list(data.keys())}\n")

# 2. Access the BitArray
# 'meas' is the default classical register name created by measure_all()
bit_array = data.meas
print(f"BitArray type: {type(bit_array)}")
print(f"BitArray shape: {bit_array.array.shape}")

# 3. View raw measurement data
for i in range(min(10, bit_array.array.shape[0])):
    print(f"   Shot {i}: {bit_array.array[i]}")

# 4. Convert to counts dictionary
counts = bit_array.get_counts()
sorted_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
print(f"\nMeasurement counts (all outcomes):")
for outcome, count in sorted_counts:
    print(f"   {outcome}: {count} shots ({count/1000*100:.1f}%)")

In [ ]:
# optionally, convert away from the native BitArray format to a dictionary format
counts = data.meas.get_counts()
print(f"Counts: {counts}")

## Objective 4 : Run Job sessions

Sessions provide an efficient way to run multiple related jobs on the same quantum hardware. They maintain context between jobs, which can reduce overhead and improve performance for iterative algorithms.


A **Session** is a context that:
1. **Reserves hardware time** for multiple jobs
2. **Maintains circuit context** between jobs
3. **Reduces overhead** for iterative algorithms
4. **Ensures consistent hardware conditions** for related jobs

**Session Lifecycle:**
1. **Open**: Create a session with a specific backend
2. **Use**: Run multiple jobs within the session
3. **Close**: Explicitly close or let timeout

**Key Parameters:**
- **TTL (Time To Live)**: Maximum session duration (default: 8 hours)
- **Max Time**: Maximum job execution time within session


<span style="color:red"> **Note:Users on Open Plan can not run job in session execution mode, The following code blocks in the rest of the notebook will fail** </span>

### Open a session

Can be done by initializing the `Session` class or by using Context Manager `with Session()`

In [ ]:
from qiskit_ibm_runtime import (
    Session,
    SamplerV2 as Sampler,
    EstimatorV2 as Estimator,
)

session = Session(backend=backend)
estimator = Estimator(mode=session)
sampler = Sampler(mode=session)
# Close the session because no context manager was used.
session.close()

In [ ]:
from qiskit_ibm_runtime import (
    Session,
    SamplerV2 as Sampler,
    EstimatorV2 as Estimator,
)
 
backend = service.least_busy(operational=True, simulator=False)
with Session(backend=backend):
    estimator = Estimator()
    sampler = Sampler()

### Session Length




Similar to `Batch`, A session max TTL can be defined with `max_time` , it starts when the session starts, when max time is reached , the session is closed and any job running will finish but all queued jobs will fail.

| Plan Type | Maximum Job Duration |
|-----------|-----------------------|
| Paid Plans | 8 hours |

Additionlly there is also interactive TTL can not be configured (1 minute), if there is no session jobs on the queue within the window, the session is temporarliy deactivated

### End a session

A session is closed when any of the following happens

* TTL is reached
* session is manually canceled or closed using `session.close()`
* session is used in a context manager , automatically closed when the context ends

When a session is closed. it no longer accepts jobs, however the already submitted jobs will be executed.


In [ ]:
session = Session(backend=backend)
 
estimator = Estimator(mode=session)
sampler = Sampler(mode=session)
job1 = estimator.run([estimator_pub])
job2 = sampler.run([sampler_pub])
print(f"Result1: {job1.result()}")
print(f"Result2: {job2.result()}")
 
# Manually close the session. Running and queued jobs will run to completion.
session.close()

### Check Session status

Use <code>session.status()</code> to check the session status, it can be one of the following

* Pending
* In Progress, accepting new jobs
* In Progress, not accepting new jobs : submitted jobs will be completed, and session will be closed after that
* Closed

### Determine Session details

In [ ]:
from qiskit_ibm_runtime import (
    QiskitRuntimeService,
    Session,
    EstimatorV2 as Estimator,
)
  
with Session(backend=backend) as session:
    print(session.details())

### Usage Patterns

Sessions are most useful for algorithms that requitr communication between classical and quantum resources.
for example

* Run iterative workload that uses SCiPy to minimize cost function
* Run VQE algorithms in sessions

---

## Practice Questions

**1)**

**Answer:**
<details> <br/>

</details>